In [3]:
from ReconstructOrder.compute.reconstruct import ImgReconstructor
from ReconstructOrder.datastructures import PhysicalData
from ReconstructOrder.datastructures.intensity_data import IntensityData
import numpy as np
import tifffile as tf
import os

%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
plt.interactive(False)



## Assigning Intensity sample data

In [ ]:
path_to_sample = ""

int_dat = IntensityData()

# if you want to call images by their channel name, assign channel names here
int_dat.channel_names = ['IExt', 'I90', 'I135', 'I45', 'I0']

int_dat.replace_image(tf.imread(os.path.join(path_to_sample, 'img_000000010_state0_002.tif')), "IExt")
int_dat.replace_image(tf.imread(os.path.join(path_to_sample, 'img_000000010_state1_002.tif')), "I90")
int_dat.replace_image(tf.imread(os.path.join(path_to_sample, 'img_000000010_state2_002.tif')), "I135")
int_dat.replace_image(tf.imread(os.path.join(path_to_sample, 'img_000000010_state3_002.tif')), "I45")
int_dat.replace_image(tf.imread(os.path.join(path_to_sample, 'img_000000010_state4_002.tif')), "I0")


## Assigning Intensity Background data

In [ ]:
path_to_background = ""

bg_int = IntensityData()
bg_int.channel_names = ['IExt', 'I90', 'I135', 'I45', 'I0']

int_dat.add_image(tf.imread(os.path.join(path_to_background, 'img_000000010_state0_002.tif')))
int_dat.add_image(tf.imread(os.path.join(path_to_background, 'img_000000010_state1_002.tif')))
int_dat.add_image(tf.imread(os.path.join(path_to_background, 'img_000000010_state2_002.tif')))
int_dat.add_image(tf.imread(os.path.join(path_to_background, 'img_000000010_state3_002.tif')))
int_dat.add_image(tf.imread(os.path.join(path_to_background, 'img_000000010_state4_002.tif')))

## Compute background stokes, normalize it

In [ ]:
img_reconstructor = ImgReconstructor(bg_int.data.shape,
                                     bg_method="Local_fit",
                                     n_slice_local_bg=1,
                                     poly_fit_order=2,
                                     swing=0.03,
                                     wavelength=532,
                                     azimuth_offset=0,
                                     circularity='rcp')

background_stokes = img_reconstructor.compute_stokes(bg_int)
background_normalized = img_reconstructor.stokes_normalization(background_stokes)


## Compute sample stokes, normalize it

In [ ]:
print('computing stokes')
sample_stokes = img_reconstructor.compute_stokes(int_dat)
sample_normalized = img_reconstructor.stokes_normalization(sample_stokes)


>
>### Correct for background using several methods


In [ ]:
# local_fit
img_reconstructor.bg_method = "Local_fit"
sample_bg_corrected_local_fit = img_reconstructor.correct_background(sample_normalized, background_normalized)

# local_filter
img_reconstructor.bg_method = "Local_filter"
sample_bg_corrected_local_filter = img_reconstructor.correct_background(sample_normalized, background_normalized)

# standard method
img_reconstructor.bg_method = None
sample_bg_corrected_global = img_reconstructor.correct_background(sample_normalized, background_normalized)


>
>### Compute physical


In [ ]:
sample_physical_local_fit = img_reconstructor.reconstruct_birefringence(sample_bg_corrected_local_fit)

sample_physical_local_filter = img_reconstructor.reconstruct_birefringence(sample_bg_corrected_local_filter)

sample_physical_global = img_reconstructor.reconstruct_birefringence(sample_bg_corrected_global)


## Plot physical

In [ ]:
xyextent=[-1.5,1.5,-1.5,1.5]

fig1 = plt.figure(figsize=(20,20)) # create a figure with the default size 

ax1 = fig1.add_subplot(2,2,1) 
im=plt.imshow(sample_physical_local_fit.retardance,cmap='gray',extent=xyextent, origin='lower');
plt.title('retardance')
ax1.figure.colorbar(im);

ax2=fig1.add_subplot(2,2,2)
im=plt.imshow(sample_physical_local_fit.slowaxis,cmap='twilight', extent=xyextent, origin='lower'); # Need to adapt this look up table to represent orientation.
plt.title('slow axis')
ax2.figure.colorbar(im);

ax3=fig1.add_subplot(2,2,3)
im=plt.imshow(sample_physical_local_fit.transmission,cmap='gray',extent=xyextent, origin='lower');
plt.title('Brightfield_computed')
ax3.figure.colorbar(im);

ax4=fig1.add_subplot(2,2,4)
im=plt.imshow(sample_physical_local_fit.polarization,cmap='gray',extent=xyextent, origin='lower', vmin=0.9, vmax=1.1);
plt.title('polarization')
ax4.figure.colorbar(im);


## Plot normalized stokes

In [ ]:
fig2 = plt.figure(figsize=(20,20)) # create a figure with the default size 

ax1 = fig2.add_subplot(2,2,1) 
im=plt.imshow(sample_normalized.s0,cmap='gray',extent=xyextent, origin='lower');
plt.title('s0')
ax1.figure.colorbar(im);

ax2=fig2.add_subplot(2,2,2)
im=plt.imshow(sample_normalized.s1,cmap='gray',extent=xyextent, origin='lower'); # Need to adapt this look up table to represent orientation.
plt.title('s1')
ax2.figure.colorbar(im);

ax3=fig2.add_subplot(2,2,3)
im=plt.imshow(sample_normalized.s2,cmap='gray',extent=xyextent, origin='lower');
plt.title('s2')
ax3.figure.colorbar(im);

ax4=fig2.add_subplot(2,2,4)
im=plt.imshow(sample_normalized.s3,cmap='gray',extent=xyextent, origin='lower');
plt.title('s3')
ax4.figure.colorbar(im);